In [76]:
# load packages
import numpy as np 
from scipy.io   import  loadmat
from scipy import stats
import pandas as pd
import os
import matplotlib.pyplot as plt #import matplotlib as plt
from scipy.optimize import curve_fit 
import seaborn as sns #import mat73
import pickle as pkl
from datetime import datetime
import statsmodels.api as sm 
from statsmodels.formula.api import ols

In [77]:
#create data
df = pd.DataFrame({'water': np.repeat(['daily', 'weekly'], 15),
                   'sun': np.tile(np.repeat(['low', 'med', 'high'], 5), 2),
                   'height': [6, 6, 6, 5, 6, 5, 5, 6, 4, 5,
                              6, 6, 7, 8, 7, 3, 4, 4, 4, 5,
                              4, 4, 4, 4, 4, 5, 6, 6, 7, 8]})

In [78]:
print(df)

     water   sun  height
0    daily   low       6
1    daily   low       6
2    daily   low       6
3    daily   low       5
4    daily   low       6
5    daily   med       5
6    daily   med       5
7    daily   med       6
8    daily   med       4
9    daily   med       5
10   daily  high       6
11   daily  high       6
12   daily  high       7
13   daily  high       8
14   daily  high       7
15  weekly   low       3
16  weekly   low       4
17  weekly   low       4
18  weekly   low       4
19  weekly   low       5
20  weekly   med       4
21  weekly   med       4
22  weekly   med       4
23  weekly   med       4
24  weekly   med       4
25  weekly  high       5
26  weekly  high       6
27  weekly  high       6
28  weekly  high       7
29  weekly  high       8


In [79]:
#perform two-way ANOVA
model = ols('height ~ C(water) + C(sun) + C(water):C(sun)', data=df).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(water),8.533333,1.0,16.0000,0.000527
C(sun),24.866667,2.0,23.3125,0.000002
C(water):C(sun),2.466667,2.0,2.3125,0.120667
Residual,12.800000,24.0,NaN,NaN


#### import and set actual anova params

In [80]:
# Main Directory of processed file from MatLab
#MainDir = 'D:\\AttnXV3_analysis\\RCA_F1\\AvgCRFs\\' # set dir
MainDir = 'C:\\plimon\\LTP_analysis\\RCA_F1\\AmpResponseDiffs\\' # set dir
os.chdir(MainDir) # change old dir, to this dir
d = os.listdir(MainDir) # list files in dir
print(f'Files on hand: {d}')
##############################################
FileN = d[0] # choose one                        
file_path1 = os.path.join(MainDir, FileN) # join paths and prep 2 load
print('Current WD:',file_path1) # does path exist ... ?
print('Does File #1 Exist?',os.path.exists(file_path1)) # yes or no

Files on hand: ['LTP_ResponseDifferences_20240223_1325.pkl', 'PostPreResponseDifferences_20240220_1029.pkl']
Current WD: C:\plimon\LTP_analysis\RCA_F1\AmpResponseDiffs\LTP_ResponseDifferences_20240223_1325.pkl
Does File #1 Exist? True


In [81]:
loadData = pkl.load(open(file_path1,'rb'))
print(loadData.keys())

dict_keys(['DiffData', 'ErrVals', 'SubNames', 'DictMainKeys', 'ContLevs', 'crfLabs', 'VoiceMemo'])


In [82]:
data = loadData['DiffData']
# wont load error bars just yet
data_labels = loadData['DictMainKeys']
SubNames = np.array(loadData['SubNames'])
note = loadData['VoiceMemo']
contrast_levels = loadData['ContLevs']
contrast_labs = loadData['crfLabs']
print(note)
NumSubs = int(len(SubNames))
print(data_labels) # labels for nr plots

['Cont:Post - pre responses for all conditions for all subs, sub x contrast diff']
['attnL F1', 'attnL F2', 'attnR F1', 'attnR F2']


#### Stack all condition differences for 1 subject - make into a function

In [83]:
NumBins = 6 # number of contrasts
NumHarms = 2 # number of harmonic data: 2F1, 4F1
NumComp = 0 # first component from RCA
NumConds = 4

In [84]:
RowPop = NumBins * NumHarms * NumConds # 6 x 2 x 4
SubRows = NumBins*NumHarms
ContrastInd = np.arange(0,NumBins)
ContrastArr = np.arange(RowPop) % NumBins

In [85]:
AttnXArr = np.zeros((RowPop))
FreqFiltArr = np.zeros((RowPop))
HarmArr = np.zeros((RowPop))
# load subject response differnces across conditions
for co in range(NumConds):
    HarmArr[(co*SubRows):(co+1)*SubRows]  = [0] * NumBins + [1] * NumBins # index harmonic data:  2F / 4F 
if co == 0 or co == 1: # Index if Data is AttnL or AttnR
    AttnXArr[(co*SubRows):(co+1)*(SubRows)] = 0
else:
    AttnXArr[(co*SubRows):(co+1)*(SubRows)] = 1
if co % 2:          # Index what data was F1 = 6 Hz or F2 = 7.5 Hz 
    FreqFiltArr[(co*SubRows):(co+1)*SubRows] = [1]*SubRows
else:
    FreqFiltArr[(co*SubRows):(co+1)*SubRows] = [0]*SubRows

In [86]:
def MakeDataFramePerSubject(RowPop,SubRows,ContrastArr,AttnXArr,FreqFiltArr,HarmArr,NumConds,soi, dictIn):
    SubjArr = [soi] * RowPop # Subject label
    ValArr = np.zeros((RowPop)) # INDEPENDENT VARIABLE 12 x 4  = 48 rows per val
    # load subject response differnces across conditions
    for co in range(NumConds):
        dIn = dictIn[co][soi,:] 
        ValArr[(co*SubRows):(co+1)*SubRows] = dIn # Sort Data Values 
    
    dataOut = {'Subj': SubjArr,
                'AttnX': AttnXArr,
                'FreqFilt': FreqFiltArr,
                'Harmonic': HarmArr,
                'Contrast': ContrastArr,
                'Vals': ValArr}
    df = pd.DataFrame(dataOut)
    return df

In [87]:
Sub_df = {}
for su in range(NumSubs):
    Sub_df[su]= MakeDataFramePerSubject(RowPop,SubRows,ContrastArr,AttnXArr,FreqFiltArr,HarmArr,NumConds,soi=su,dictIn=data)

#### Load Some DateFrames to see if they differ

In [88]:
print(Sub_df[22])

    Subj  AttnX  FreqFilt  Harmonic  Contrast      Vals
0     22    0.0       0.0       0.0         0  0.115997
1     22    0.0       0.0       0.0         1 -0.035183
2     22    0.0       0.0       0.0         2 -0.001178
3     22    0.0       0.0       0.0         3  0.020009
4     22    0.0       0.0       0.0         4  0.218810
5     22    0.0       0.0       0.0         5  0.823410
6     22    0.0       0.0       1.0         0 -0.474678
7     22    0.0       0.0       1.0         1  0.201052
8     22    0.0       0.0       1.0         2  0.060527
9     22    0.0       0.0       1.0         3  0.622315
10    22    0.0       0.0       1.0         4 -0.057761
11    22    0.0       0.0       1.0         5  0.134541
12    22    0.0       0.0       0.0         0 -0.246698
13    22    0.0       0.0       0.0         1  0.643388
14    22    0.0       0.0       0.0         2  0.767005
15    22    0.0       0.0       0.0         3  1.058887
16    22    0.0       0.0       0.0         4  1

In [89]:
anova_df = np.zeros((RowPop*NumSubs,6))
print(anova_df.shape)

headerArr = Sub_df[0].columns.values
print(headerArr)

(1200, 6)
['Subj' 'AttnX' 'FreqFilt' 'Harmonic' 'Contrast' 'Vals']


In [90]:
for su in range(NumSubs):
    aIn = np.array(Sub_df[su])
    anova_df[(RowPop*su):(RowPop*(su+1)),:] = aIn

In [91]:
anovaDataArr = np.append([headerArr],anova_df, axis = 0 )
anova_dataf = pd.DataFrame(anovaDataArr)

In [92]:
anova_dataf

,0,1,2,3,4,5
0,Subj,AttnX,FreqFilt,Harmonic,Contrast,Vals
1,0.0,0.0,0.0,0.0,0.0,-0.053774
2,0.0,0.0,0.0,0.0,1.0,-0.395514
3,0.0,0.0,0.0,0.0,2.0,0.293846
4,0.0,0.0,0.0,0.0,3.0,0.278024
...,...,...,...,...,...,...
1196,24.0,1.0,1.0,1.0,1.0,-0.210037
1197,24.0,1.0,1.0,1.0,2.0,0.035446
1198,24.0,1.0,1.0,1.0,3.0,0.010592
1199,24.0,1.0,1.0,1.0,4.0,0.058085
